# Bring Your Own Model (k-means)

Reference:  
https://github.com/awslabs/amazon-sagemaker-examples/blob/master/advanced_functionality/kmeans_bring_your_own_model/kmeans_bring_your_own_model.ipynb



In [2]:
import sys
!conda install --yes --prefix {sys.prefix} mxnet

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.12
  latest version: 4.8.2

Please update conda by running

    $ conda update -n base conda



## Package Plan ##

  environment location: /home/ec2-user/anaconda3/envs/mxnet_p36

  added / updated specs: 
    - mxnet


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    xorg-libice-1.0.10         |       h516909a_0          57 KB  conda-forge
    harfbuzz-1.9.0             |    he243708_1001         957 KB  conda-forge
    xorg-xextproto-7.3.0       |    h14c3975_1002          27 KB  conda-forge
    blas-2.14                  |         openblas          10 KB  conda-forge
    x264-1!152.20180806        |       h14c3975_0         1.4 MB  conda-forge
    xorg-libx11-1.6.9          |       h516909a_0         918 KB  conda-forge
    jasper-2.0.14              |       habb8e15_1         1.2 MB  

In [3]:
import mxnet as mx

In [7]:
import numpy as np
import sklearn.cluster
import pickle
import gzip
import urllib.request
import json
import mxnet as mx
import boto3
import time
import io
import os

In [8]:
import boto3
import re
 
from sagemaker import get_execution_role

In [9]:
bucket = 'md-ml-labs-bucket'
prefix = 'sage-02-kmeans-byoc'

In [29]:
# use this if running sagemaker locally
def resolve_sm_role():
    client = boto3.client('iam', region_name='us-east-2')
    response_roles = client.list_roles(
        PathPrefix='/',
        # Marker='string',
        MaxItems=999
    )
    for role in response_roles['Roles']:
        if role['RoleName'].startswith('AmazonSageMaker-ExecutionRole-'):
            print('Resolved SageMaker IAM Role to: ' + str(role))
            return role['Arn']
    raise Exception('Could not resolve what should be the SageMaker role to be used')

# this is the role created by sagemaker notebook on aws
role_arn = resolve_sm_role()
role = role_arn
print(role_arn)


Resolved SageMaker IAM Role to: {'Path': '/service-role/', 'RoleName': 'AmazonSageMaker-ExecutionRole-20200201T085431', 'RoleId': 'AROA4UGSQ27F7UPXXYEND', 'Arn': 'arn:aws:iam::868024899531:role/service-role/AmazonSageMaker-ExecutionRole-20200201T085431', 'CreateDate': datetime.datetime(2020, 2, 1, 14, 54, 12, tzinfo=tzlocal()), 'AssumeRolePolicyDocument': {'Version': '2012-10-17', 'Statement': [{'Effect': 'Allow', 'Principal': {'Service': 'sagemaker.amazonaws.com'}, 'Action': 'sts:AssumeRole'}]}, 'Description': 'SageMaker execution role created from the SageMaker AWS Management Console.', 'MaxSessionDuration': 3600}
arn:aws:iam::868024899531:role/service-role/AmazonSageMaker-ExecutionRole-20200201T085431


# Get The Data

In [11]:
urllib.request.urlretrieve("http://deeplearning.net/data/mnist/mnist.pkl.gz", "./data/mnist.pkl.gz")
f = gzip.open('./data/mnist.pkl.gz', 'rb')

In [12]:
# pickle reference: 
# https://docs.python.org/3/library/pickle.html

# data is already organised in train, validation and testing sets
train_set, valid_set, test_set = pickle.load(f, encoding='latin1')
f.close()

In [13]:
train_set

(array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32),
 array([5, 0, 4, ..., 8, 4, 8]))

In [15]:
type(train_set)

tuple

# Train Localy

In [16]:
# sklearn.cluster.KMeans reference: 
# https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html

kmeans = sklearn.cluster.KMeans(n_clusters=10).fit(train_set[0])

# Convert to MXNet NDArray

In [17]:
# we asked to clusterize the data in 10 clusters
# data has 784 dimensions (one for each pixel in images)
# the result of kmeans is a NumPy are 10 by 784
# this is stored in kmeans.cluster_centers_

centroids = mx.ndarray.array(kmeans.cluster_centers_)

In [19]:
[centroids]

[
 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]
 <NDArray 10x784 @cpu(0)>]

In [17]:
# model_algo-1 is a standardised name where model should be saved
mx.ndarray.save('./data-kmeans/model_algo-1', [centroids])

In [20]:
!tar czvf ./data-kmeans/model.tar.gz ./data-kmeans/model_algo-1 

./data-kmeans/model_algo-1


In [22]:
# upload to s3
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'model.tar.gz')).upload_file('./data/model.tar.gz')

# Host

In [23]:
# create a name for our model host container that should be unique

from sagemaker.amazon.amazon_estimator import get_image_uri
kmeans_model = 'DEMO-kmeans-byom-' + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())
kmeans_model

'DEMO-kmeans-byom-2020-02-01-22-38-25'

In [25]:
sm = boto3.client('sagemaker')

# reference get_image_uri:
# https://github.com/aws/sagemaker-python-sdk/blob/master/src/sagemaker/amazon/amazon_estimator.py
# description: Return algorithm image URI for the given AWS region, repository name, and repository version
container = get_image_uri(boto3.Session().region_name, 'kmeans')
container

'404615174143.dkr.ecr.us-east-2.amazonaws.com/kmeans:1'

In [30]:
# reference create_model:
# https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.create_model

# create hosting inference for model
create_model_response = sm.create_model(
    ModelName=kmeans_model,
    ExecutionRoleArn=role,
    PrimaryContainer={
        'Image': container,
        'ModelDataUrl': 's3://{}/{}/model.tar.gz'.format(bucket, prefix)})

print(create_model_response['ModelArn'])

arn:aws:sagemaker:us-east-2:868024899531:model/demo-kmeans-byom-2020-02-01-22-38-25


In [31]:
kmeans_endpoint_config = 'DEMO-kmeans-byom-endpoint-config-' + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())
print(kmeans_endpoint_config)

DEMO-kmeans-byom-endpoint-config-2020-02-01-23-04-22


In [32]:
create_endpoint_config_response = sm.create_endpoint_config(
    EndpointConfigName=kmeans_endpoint_config,
    ProductionVariants=[{
        'InstanceType': 'ml.m4.xlarge',
        'InitialInstanceCount': 1,
        'ModelName': kmeans_model,
        'VariantName': 'AllTraffic'}])

In [34]:
print(create_endpoint_config_response)

{'EndpointConfigArn': 'arn:aws:sagemaker:us-east-2:868024899531:endpoint-config/demo-kmeans-byom-endpoint-config-2020-02-01-23-04-22', 'ResponseMetadata': {'RequestId': '6bc29094-76a4-4d73-b39f-c43ce0466599', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '6bc29094-76a4-4d73-b39f-c43ce0466599', 'content-type': 'application/x-amz-json-1.1', 'content-length': '133', 'date': 'Sat, 01 Feb 2020 23:05:27 GMT'}, 'RetryAttempts': 0}}


In [35]:
print("Endpoint Config Arn: " + create_endpoint_config_response['EndpointConfigArn'])

Endpoint Config Arn: arn:aws:sagemaker:us-east-2:868024899531:endpoint-config/demo-kmeans-byom-endpoint-config-2020-02-01-23-04-22


In [41]:
kmeans_endpoint = 'DEMO-kmeans-byom-endpoint-' + time.strftime("%Y%m%d%H%M", time.gmtime())
print(kmeans_endpoint)

DEMO-kmeans-byom-endpoint-202002012325


In [42]:
# it would take a few minutes until the endpoint is up
create_endpoint_response = sm.create_endpoint(
    EndpointName=kmeans_endpoint,
    EndpointConfigName=kmeans_endpoint_config)

print(create_endpoint_response['EndpointArn'])

arn:aws:sagemaker:us-east-2:868024899531:endpoint/demo-kmeans-byom-endpoint-202002012325


In [43]:
# status will be "Creating" during the end point creation
# create_endpoint is anynchronous, ir returns imediatley, while the endpoint is in creation mode
resp = sm.describe_endpoint(EndpointName=kmeans_endpoint)
status = resp['EndpointStatus']
print("Status: " + status)

Status: Creating


In [ ]:
%%time
# we call wait because create_endpoint is an asynchronous method

# reference get_waiter:
# https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.get_waiter

sm.get_waiter('endpoint_in_service').wait(EndpointName=kmeans_endpoint)

# failed maybe because doesnt have permissions

In [ ]:
# should get here after endpoint is created
resp = sm.describe_endpoint(EndpointName=kmeans_endpoint)
status = resp['EndpointStatus']
print("Arn: " + resp['EndpointArn'])
print("Status: " + status)

if status != 'InService':
    raise Exception('Endpoint creation did not succeed')

# Validation